## Imports et Chargment des données 

In [1]:
import pandas as pd
from pathlib import Path

In [77]:
match_final_kpi = pd.read_csv(Path("../data/clean/matches_final_kpi.csv"))
dim_teams = pd.read_csv(Path("../data/clean/dim_teams.csv"))
home_stats_normalized = pd.read_csv(Path("../data/clean/home_stats_normalized.csv"))
away_stats_normalized = pd.read_csv(Path("../data/clean/away_stats_normalized.csv"))
matches_normalized = pd.read_csv(Path("../data/clean/matches_normalized.csv"))
teams_reference_normalized = pd.read_csv(Path("../data/clean/teams_reference_normalized.csv"))

print(match_final_kpi.shape, dim_teams.shape, home_stats_normalized.shape, away_stats_normalized.shape, matches_normalized.shape, teams_reference_normalized.shape)

(6861, 11) (227, 6) (6861, 4) (6861, 4) (6861, 7) (227, 2)


# Visualisation de la donnée 

In [129]:
display(dim_teams.head(),
        match_final_kpi.head(),
        home_stats_normalized.head(),
        away_stats_normalized.head(),
        matches_normalized.head(),
        teams_reference_normalized.head())

,team_canonical,team_clean_example,canonical_key,iso2,iso3,team_id
0,Afghanistan,Afghanistan,afghanistan,NaN,NaN,1
1,Albania,Albania,albania,NaN,NaN,2
2,Algeria,Algeria,algeria,NaN,NaN,3
3,American Samoa,American Samoa,americansamoa,NaN,NaN,4
4,Andorra,Andorra,andorra,NaN,NaN,5


,id_match,home_team,away_team,home_result,away_result,result,date,round,city,edition,is_final
0,1,France,Mexico,4,1,France,1930-07-13,Group,Montevideo,1930,True
1,2,Romania,Peru,3,1,Romania,1930-07-14,Group,Montevideo,1930,True
2,3,Yugoslavia,Brazil,2,1,Yugoslavia,1930-07-14,Group,Montevideo,1930,True
3,4,Argentina,France,1,0,Argentina,1930-07-15,Group,Montevideo,1930,True
4,5,Chile,Mexico,3,0,Chile,1930-07-16,Group,Montevideo,1930,True


,id_match,id_team,Number_of_goals_scored,Number_of_goals_conceded
0,1,76,4,1
1,2,163,3,1
2,3,225,2,1
3,4,10,1,0
4,5,41,3,0


,id_match,id_team,Number_of_goals_scored,Number_of_goals_conceded
0,1,130,1,4
1,2,156,1,3
2,3,28,1,2
3,4,76,0,1
4,5,130,0,3


,id_match,result,date,round,city,edition,is_final
0,1,76,1930-07-13,Group,Montevideo,1930,True
1,2,163,1930-07-14,Group,Montevideo,1930,True
2,3,225,1930-07-14,Group,Montevideo,1930,True
3,4,10,1930-07-15,Group,Montevideo,1930,True
4,5,41,1930-07-16,Group,Montevideo,1930,True


,id_team,Team_name
0,1,Afghanistan
1,2,Albania
2,3,Algeria
3,4,American Samoa
4,5,Andorra


In [130]:
display(dim_teams.shape,
        match_final_kpi.shape,
        home_stats_normalized.shape,
        away_stats_normalized.shape,
        matches_normalized.shape,
        teams_reference_normalized.shape)

(227, 6)

(6861, 11)

(6861, 4)

(6861, 4)

(6861, 7)

(227, 2)

# 📊 ANALYSE EXPLORATOIRE & DATA STORY FIFA WORLD CUP
## Histoire des Coupes du Monde à travers les données (1930-2022)

Cette analyse révèle **92 ans d'histoire footballistique** à travers les matchs, buts, et performances des équipes nationales.

In [79]:
# Import des bibliothèques de visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Configuration globale des graphiques
plt.style.use('default')
sns.set_palette("husl")
px.defaults.template = "plotly_white"

print("✅ Bibliothèques de visualisation chargées")

✅ Bibliothèques de visualisation chargées


## 🌍 Vue d'ensemble - L'évolution du football mondial
### Combien de matchs, de buts, d'équipes ? Que nous disent les chiffres globaux ?

In [104]:
# 📈 VISUALISATION 1 : Évolution du nombre de matchs par Coupe du Monde
matches_by_edition = match_final_kpi[(match_final_kpi["is_final"]== True)&(match_final_kpi["date"].notna())].groupby('edition').size().reset_index(name='nb_matches')
matches_by_edition['edition'] = matches_by_edition['edition'].astype(int)

fig = px.bar(matches_by_edition.sort_values('edition'), 
             x='edition', y='nb_matches',
             title="🏆 Évolution du nombre de matchs par Coupe du Monde (1930-2022)",
             labels={'edition': 'Année de la Coupe du Monde', 'nb_matches': 'Nombre de matchs'},
             color='nb_matches',
             color_continuous_scale='viridis')

fig.update_layout(height=500)
fig.show()

print(f"📊 Insight: De {matches_by_edition['nb_matches'].min()} matchs en 1930 à {matches_by_edition['nb_matches'].max()} matchs au maximum")

📊 Insight: De 15 matchs en 1930 à 64 matchs au maximum


In [106]:
# 🎯 VISUALISATION 2 : Total des buts par édition - L'explosion offensive
# Calcul des buts totaux par match puis par édition
goals_by_match = pd.merge(matches_normalized[(matches_normalized["is_final"]==True)&(matches_normalized["date"].notna())], home_stats_normalized, on='id_match', how='inner')
goals_by_match = pd.merge(goals_by_match, away_stats_normalized, on='id_match', suffixes=('_home', '_away'), how='inner')
goals_by_match['total_goals'] = goals_by_match['Number_of_goals_scored_home'] + goals_by_match['Number_of_goals_scored_away']

goals_by_edition = goals_by_match.groupby('edition').agg({
    'total_goals': 'sum',
    'id_match': 'count'
}).reset_index()
goals_by_edition['avg_goals_per_match'] = goals_by_edition['total_goals'] / goals_by_edition['id_match']
goals_by_edition['edition'] = goals_by_edition['edition'].astype(int)

# Graphique double axe
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=goals_by_edition['edition'], y=goals_by_edition['total_goals'], 
                     name="Total buts", marker_color='lightblue'), secondary_y=False)

fig.add_trace(go.Scatter(x=goals_by_edition['edition'], y=goals_by_edition['avg_goals_per_match'], 
                         mode='lines+markers', name="Moyenne buts/match", 
                         line=dict(color='red', width=3)), secondary_y=True)

fig.update_xaxes(title_text="Année")
fig.update_yaxes(title_text="Total buts", secondary_y=False)
fig.update_yaxes(title_text="Buts par match", secondary_y=True)
fig.update_layout(title="⚽ Évolution des buts : Total vs Moyenne par match")
fig.show()

print(f"🎯 L'édition la plus prolifique: {goals_by_edition.loc[goals_by_edition['total_goals'].idxmax(), 'edition']} avec {goals_by_edition['total_goals'].max()} buts")

🎯 L'édition la plus prolifique: 2022 avec 172 buts


## 👑 Les légendes du football - Top équipes
### Qui sont les vrais maîtres des Coupes du Monde ? Analyse des performances par équipe

In [109]:
# 👑 VISUALISATION 4 : TOP 15 des équipes les plus actives
# Calcul des statistiques par équipe
all_team_stats = []

# Statistiques domicile
home_team_stats = home_stats_normalized.merge(teams_reference_normalized, 
                                            left_on='id_team', right_on='id_team')
home_team_stats['location'] = 'home'

# Statistiques extérieur  
away_team_stats = away_stats_normalized.merge(teams_reference_normalized,
                                            left_on='id_team', right_on='id_team')
away_team_stats['location'] = 'away'

# Combinaison des statistiques
combined_stats = pd.concat([
    home_team_stats[['Team_name', 'Number_of_goals_scored', 'Number_of_goals_conceded', 'location','id_match']],
    away_team_stats[['Team_name', 'Number_of_goals_scored', 'Number_of_goals_conceded', 'location']]
])
combined_stats = pd.merge(
    combined_stats,
    matches_normalized[(matches_normalized["is_final"]==True)&(matches_normalized["date"].notna())], 
    on='id_match',
    how='inner'
)
team_summary = combined_stats.groupby('Team_name').agg({
    'Number_of_goals_scored': 'sum',
    'Number_of_goals_conceded': 'sum',
    'Team_name': 'count'  # Nombre de matchs
}).rename(columns={'Team_name': 'total_matches'})

team_summary['goal_difference'] = team_summary['Number_of_goals_scored'] - team_summary['Number_of_goals_conceded']
team_summary = team_summary.sort_values('total_matches', ascending=False).head(15)

# Graphique horizontal moderne
fig = px.bar(team_summary.reset_index().head(15), 
             x='total_matches', y='Team_name', 
             orientation='h',
             title="🏆 Top 15 des équipes les plus actives en Coupe du Monde",
             labels={'total_matches': 'Nombre de matchs joués', 'Team_name': 'Équipe'},
             color='goal_difference',
             color_continuous_scale='RdYlGn',
             text='total_matches')

fig.update_traces(texttemplate='%{text}', textposition='inside')
fig.update_layout(height=600, yaxis={'categoryorder':'total ascending'})
fig.show()

print("🎯 Top 3 des équipes les plus actives:")
for i, (team, data) in enumerate(team_summary.head(3).iterrows()):
    print(f"   {i+1}. {team}: {data['total_matches']} matchs, {data['Number_of_goals_scored']} buts marqués")

🎯 Top 3 des équipes les plus actives:
   1. Brazil: 78 matchs, 178 buts marqués
   2. Italy: 50 matchs, 82 buts marqués
   3. Argentina: 49 matchs, 92 buts marqués


In [110]:
# ⚔️ VISUALISATION 5 : Scatter Plot - Efficacité offensive vs défensive
# Analyse de l'efficacité des équipes (top 20)
top_teams = team_summary.head(20).reset_index()

fig = px.scatter(top_teams, 
                x='Number_of_goals_scored', 
                y='Number_of_goals_conceded',
                size='total_matches',
                color='goal_difference',
                hover_name='Team_name',
                title="⚔️ Efficacité des équipes : Attaque vs Défense",
                labels={
                    'Number_of_goals_scored': 'Buts marqués', 
                    'Number_of_goals_conceded': 'Buts encaissés',
                    'goal_difference': 'Différence de buts'
                },
                color_continuous_scale='RdYlGn',
                size_max=60)

# Ligne de référence (performance équilibrée)
fig.add_shape(type="line", x0=0, y0=0, 
              x1=top_teams['Number_of_goals_scored'].max(), 
              y1=top_teams['Number_of_goals_scored'].max(),
              line=dict(color="gray", dash="dash", width=2))

fig.update_layout(height=600)
fig.show()

print("💡 Lecture du graphique:")
print("   🟢 Équipes en vert (au-dessus de la ligne): Plus de buts marqués qu'encaissés")
print("   🔴 Équipes en rouge (en-dessous de la ligne): Plus de buts encaissés que marqués")
print("   📊 Taille des bulles = Nombre de matchs joués")

💡 Lecture du graphique:
   🟢 Équipes en vert (au-dessus de la ligne): Plus de buts marqués qu'encaissés
   🔴 Équipes en rouge (en-dessous de la ligne): Plus de buts encaissés que marqués
   📊 Taille des bulles = Nombre de matchs joués


In [111]:
# 🇫🇷 ANALYSE SPÉCIALE : Performance de l'équipe de France
print("🇫🇷 ANALYSE ÉQUIPE DE FRANCE - COUPE DU MONDE")
print("=" * 55)

# Filtrer les données pour la France uniquement
france_stats = combined_stats[(combined_stats['Team_name'] == 'France')].copy()

if len(france_stats) > 0:
    # Statistiques générales
    total_matches_france = len(france_stats)
    total_goals_scored = france_stats['Number_of_goals_scored'].sum()
    total_goals_conceded = france_stats['Number_of_goals_conceded'].sum()
    goal_difference = total_goals_scored - total_goals_conceded
    
    # Calcul des victoires, nuls, défaites
    # On récupère les matchs avec résultats détaillés
    france_detailed = []
    for _, match in france_stats.iterrows():
        match_id = match['id_match']
        # Récupérer le score du match
        home_stat = home_stats_normalized[home_stats_normalized['id_match'] == match_id]
        away_stat = away_stats_normalized[away_stats_normalized['id_match'] == match_id]
        
        if not home_stat.empty and not away_stat.empty:
            home_goals = home_stat['Number_of_goals_scored'].iloc[0]
            away_goals = away_stat['Number_of_goals_scored'].iloc[0]
            
            # Déterminer si la France était à domicile ou extérieur
            if match['location'] == 'home':
                france_goals = home_goals
                opponent_goals = away_goals
            else:
                france_goals = away_goals
                opponent_goals = home_goals
            
            if france_goals > opponent_goals:
                result = 'Victoire'
            elif france_goals < opponent_goals:
                result = 'Défaite'
            else:
                result = 'Match Nul'
                
            france_detailed.append({
                'result': result,
                'france_goals': france_goals,
                'opponent_goals': opponent_goals
            })
    
    # Compter les résultats
    france_results = pd.DataFrame(france_detailed)
    if not france_results.empty:
        victories = len(france_results[france_results['result'] == 'Victoire'])
        draws = len(france_results[france_results['result'] == 'Match Nul'])
        defeats = len(france_results[france_results['result'] == 'Défaite'])
        
        # Pourcentage de réussite (victoires + nuls)
        success_rate = ((victories + draws) / total_matches_france) * 100
        victory_rate = (victories / total_matches_france) * 100
    else:
        victories = draws = defeats = 0
        success_rate = victory_rate = 0
    
    print(f"📊 STATISTIQUES GÉNÉRALES")
    print(f"   • Nombre total de matchs joués : {total_matches_france}")
    print(f"   • Participations en Coupe du Monde analysées")
    
    print(f"\n🏆 BILAN SPORTIF")
    print(f"   • Victoires : {victories} ({victory_rate:.1f}%)")
    print(f"   • Matchs nuls : {draws} ({(draws/total_matches_france)*100 if total_matches_france > 0 else 0:.1f}%)")
    print(f"   • Défaites : {defeats} ({(defeats/total_matches_france)*100 if total_matches_france > 0 else 0:.1f}%)")
    
    print(f"\n⚽ STATISTIQUES OFFENSIVES/DÉFENSIVES")
    print(f"   • Buts marqués : {total_goals_scored}")
    print(f"   • Buts encaissés : {total_goals_conceded}")
    print(f"   • Différence de buts : {goal_difference:+d}")
    print(f"   • Moyenne buts marqués/match : {total_goals_scored/total_matches_france:.2f}")
    print(f"   • Moyenne buts encaissés/match : {total_goals_conceded/total_matches_france:.2f}")
    
    print(f"\n🎯 TAUX DE RÉUSSITE")
    print(f"   • Pourcentage de victoires : {victory_rate:.1f}%")
    print(f"   • Pourcentage de réussite (V+N) : {success_rate:.1f}%")
    
    # Conclusion dynamique
    print(f"\n💡 CONCLUSION")
    if goal_difference > 0:
        offensive_performance = "solide attaque"
    elif goal_difference == 0:
        offensive_performance = "équilibre offensif-défensif"
    else:
        offensive_performance = "défense à améliorer"
    
    if victory_rate > 50:
        overall_performance = "excellente performance"
    elif victory_rate > 33:
        overall_performance = "performance honorable"
    else:
        overall_performance = "marge de progression importante"
    
    print(f"   🇫🇷 La France affiche une {overall_performance} en Coupe du Monde")
    print(f"   📈 Avec {total_matches_france} matchs et une {offensive_performance}")
    print(f"   🏆 Taux de réussite de {success_rate:.1f}% (victoires + nuls)")
    
    if goal_difference > 10:
        print(f"   ⭐ Différentiel de +{goal_difference} buts témoigne d'une grande efficacité")
    elif goal_difference > 0:
        print(f"   ✅ Différentiel positif de +{goal_difference} buts, signe d'une équipe compétitive")
    elif goal_difference == 0:
        print(f"   ⚖️ Équilibre parfait entre attaque et défense")
    else:
        print(f"   ⚠️ Différentiel négatif de {goal_difference} buts, axes d'amélioration identifiés")

else:
    print("❌ Aucune donnée trouvée pour l'équipe de France")
    print("   Vérifier l'orthographe : 'France' dans les données")

🇫🇷 ANALYSE ÉQUIPE DE FRANCE - COUPE DU MONDE
📊 STATISTIQUES GÉNÉRALES
   • Nombre total de matchs joués : 35
   • Participations en Coupe du Monde analysées

🏆 BILAN SPORTIF
   • Victoires : 26 (74.3%)
   • Matchs nuls : 5 (14.3%)
   • Défaites : 4 (11.4%)

⚽ STATISTIQUES OFFENSIVES/DÉFENSIVES
   • Buts marqués : 82
   • Buts encaissés : 33
   • Différence de buts : +49
   • Moyenne buts marqués/match : 2.34
   • Moyenne buts encaissés/match : 0.94

🎯 TAUX DE RÉUSSITE
   • Pourcentage de victoires : 74.3%
   • Pourcentage de réussite (V+N) : 88.6%

💡 CONCLUSION
   🇫🇷 La France affiche une excellente performance en Coupe du Monde
   📈 Avec 35 matchs et une solide attaque
   🏆 Taux de réussite de 88.6% (victoires + nuls)
   ⭐ Différentiel de +49 buts témoigne d'une grande efficacité
